<a href="https://colab.research.google.com/github/Taaniya/exploring-gpt2-language-model/blob/main/Exploring_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.1 MB/s eta 0:00:0

In [60]:
from transformers import GPT2TokenizerFast, TFGPT2LMHeadModel, GPT2LMHeadModel
from datasets import load_dataset
from transformers import pipeline
import numpy as np
import tensorflow as tf

In [4]:
tokenizer = GPT2TokenizerFast.from_pretrained('distilgpt2')

#### Model's Vocab

In [5]:
tokenizer.vocab_size

50257

In [6]:
tokenizer

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [7]:
#### Special Tokens

In [8]:
tokenizer.all_special_tokens

['<|endoftext|>']

In [9]:
tokenizer.bos_token

'<|endoftext|>'

In [10]:
tokenizer.eos_token

'<|endoftext|>'

In [11]:
tokenizer.pad_token

Using pad_token, but it is not set yet.


GPT2 doesn't use padding. It's default mex length of sentence supported is 1024

In [ ]:
tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

In [ ]:
tokenizer.model_max_length

1024

In [ ]:
! ls sample_data/

anscombe.json		      mnist_test.csv	     string_theory.txt
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
datasets = load_dataset("text", data_files={"train":["sample_data/string_theory.txt"]})

Using custom data configuration default-64e91c498735b760


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-64e91c498735b760/0.0.0/acc32f2f2ef863c93c2f30c52f7df6cc9053a1c2230b8d7da0d210404683ca08. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7
    })
})

In [ ]:
datasets['train']

Dataset({
    features: ['text'],
    num_rows: 7
})

In [ ]:
datasets['train'][:2]

{'text': ['In physics, string theory is a theoretical framework in which the point-like particles of particle physics are replaced by one-dimensional objects called strings.',
  'String theory describes how these strings propagate through space and interact with each other. ']}

In [ ]:
tokenized_datasets = datasets.map(lambda examples: tokenizer(examples['text']), batched=True, num_proc=4, remove_columns=['text'])

Parameter 'function'=<function <lambda> at 0x7fba0ceb48c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
outputs = tokenizer(datasets['train'][:2]['text'], 
                    truncation=True,
                    max_length=128, 
                    return_overflowing_tokens=True,
                    return_length=True)

In [ ]:
outputs

{'input_ids': [[818, 11887, 11, 4731, 4583, 318, 257, 16200, 9355, 287, 543, 262, 966, 12, 2339, 13166, 286, 18758, 11887, 389, 6928, 416, 530, 12, 19577, 5563, 1444, 13042, 13], [10100, 4583, 8477, 703, 777, 13042, 47933, 832, 2272, 290, 9427, 351, 1123, 584, 13, 220]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'length': [29, 16], 'overflow_to_sample_mapping': [0, 1]}

In [ ]:
print(f"input IDs length: {len(outputs['input_ids'])}")
print(f"input chunk length: {outputs['length']}")
print(f"chunk mapping: {outputs['overflow_to_sample_mapping']}")

input IDs length: 2
input chunk length: [29, 16]
chunk mapping: [0, 1]


Above 2 examples are of length 29 & 16 tokens respectively, without padding

#### Text generation

In [12]:
generator = pipeline(task='text-generation', model='distilgpt2')

This parameter is used as default value for generating sequence of max_length

In [13]:
generator.model.config.max_length

50

In [14]:
generator("how are")

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'how are you going to have to pay $100 to enter the race? Let me know in the comments below!\n\n\n- David D. Hall (D-S.D.C. – www.davidd.com)'}]

In [15]:
tokenizer("which is")

{'input_ids': [4758, 318], 'attention_mask': [1, 1]}

In [16]:
tokenizer("hello<|endoftext|>")

{'input_ids': [31373, 50256], 'attention_mask': [1, 1]}

In [17]:
tokenizer.convert_ids_to_tokens(220)

'Ġ'

In [18]:
# 505256 is the last token ID in its vocab (vocab size=50257)

tokenizer.convert_ids_to_tokens(50256)

'<|endoftext|>'

In [19]:
tokenizer.tokenize("ambigram")

['amb', 'ig', 'ram']

In [20]:
tokenizer.tokenize("In physics, string theory is a theoretical framework in which the point-like particles of \
particle physics are replaced by one-dimensional objects called strings.")

['In',
 'Ġphysics',
 ',',
 'Ġstring',
 'Ġtheory',
 'Ġis',
 'Ġa',
 'Ġtheoretical',
 'Ġframework',
 'Ġin',
 'Ġwhich',
 'Ġthe',
 'Ġpoint',
 '-',
 'like',
 'Ġparticles',
 'Ġof',
 'Ġparticle',
 'Ġphysics',
 'Ġare',
 'Ġreplaced',
 'Ġby',
 'Ġone',
 '-',
 'dimensional',
 'Ġobjects',
 'Ġcalled',
 'Ġstrings',
 '.']

This tokenizer treats spaces like parts of tokens

### Exploring its next word predictions

In [38]:
tokenized = tokenizer("how are you doing today", return_tensors='tf')

In [39]:
tokenized

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[4919,  389,  345, 1804, 1909]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

In [40]:
lm_model = TFGPT2LMHeadModel.from_pretrained("distilgpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [41]:
# set the argument to return attentions
outputs = lm_model(tokenized, output_attentions=True)

In [42]:
type(outputs)

transformers.modeling_tf_outputs.TFCausalLMOutputWithCrossAttentions

In [43]:
outputs.attentions

(<tf.Tensor: shape=(1, 12, 5, 5), dtype=float32, numpy=
 array([[[[9.99999940e-01, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00],
          [9.13400710e-01, 8.65991861e-02, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00],
          [7.97991872e-01, 1.28857613e-01, 7.31504038e-02,
           0.00000000e+00, 0.00000000e+00],
          [4.38733339e-01, 2.28085056e-01, 1.74588710e-01,
           1.58592895e-01, 0.00000000e+00],
          [4.50666547e-01, 1.83301941e-01, 1.27101392e-01,
           1.35537595e-01, 1.03392586e-01]],
 
         [[9.99999940e-01, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00],
          [7.97339362e-06, 9.99991953e-01, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00],
          [2.46466225e-05, 1.59446914e-02, 9.84030485e-01,
           0.00000000e+00, 0.00000000e+00],
          [1.85843817e-06, 1.43038947e-03, 1.16038031e-03,
           9.97407436e-01, 0.00000000e+00],
          [4.9

In [44]:
logits = outputs.logits

In [45]:
logits

<tf.Tensor: shape=(1, 5, 50257), dtype=float32, numpy=
array([[[-31.152214, -29.294815, -31.223711, ..., -41.802814,
         -41.639362, -31.38221 ],
        [-56.623074, -57.436707, -60.61329 , ..., -60.59092 ,
         -62.180145, -58.705196],
        [-65.41144 , -67.15361 , -70.58233 , ..., -73.067474,
         -73.61381 , -69.42236 ],
        [-55.67569 , -56.63195 , -60.455547, ..., -61.793   ,
         -65.24438 , -58.518982],
        [-50.469105, -51.55584 , -54.412727, ..., -62.174026,
         -61.92645 , -53.483624]]], dtype=float32)>

These logits are unnormalized scores returned by last layer of the model. Let's convert them to probabilities. When converted, the probability for every token in the input sequence, represent the probability of every word in the vocabulary that can appear following the current word. 

Let's understand this next

In [46]:
next_word_preds = tf.math.softmax(outputs.logits, axis=-1)
print(next_word_preds)

tf.Tensor(
[[[6.3007564e-04 4.0369262e-03 5.8659987e-04 ... 1.4924359e-08
   1.7574463e-08 5.0061836e-04]
  [1.5670838e-04 6.9460191e-05 2.8984364e-06 ... 2.9640032e-06
   6.0490436e-07 1.9536097e-05]
  [3.1142408e-04 5.4542674e-05 1.7687354e-06 ... 1.4735940e-07
   8.5331287e-08 5.6419531e-06]
  [2.0381255e-04 7.8330733e-05 1.7114537e-06 ... 4.4928004e-07
   1.4243042e-08 1.1868745e-05]
  [4.6291811e-04 1.5614973e-04 8.9703753e-06 ... 3.8205088e-09
   4.8937618e-09 2.2715134e-05]]], shape=(1, 5, 50257), dtype=float32)


In [47]:
# we obtain probabilities across the vocabulary for the next token at each of 5 token indices in the input sequence
next_word_preds.shape

TensorShape([1, 5, 50257])

In [49]:
# predicting 2nd word & its probability
word_idx = 1
vocab_id = np.argmax(next_word_preds[0][word_idx-1])
tokenizer.convert_ids_to_tokens([vocab_id]) , next_word_preds[0][word_idx-1].numpy()

(['ĠThe'],
 array([6.3007564e-04, 4.0369262e-03, 5.8659987e-04, ..., 1.4924359e-08,
        1.7574463e-08, 5.0061836e-04], dtype=float32))

In [48]:
# predicting 3rd word & its probability
word_idx = 2
vocab_id = np.argmax(next_word_preds[0][word_idx-1])
tokenizer.convert_ids_to_tokens([vocab_id]) , next_word_preds[0][word_idx-1].numpy()

(['Ġyou'],
 array([1.5670838e-04, 6.9460191e-05, 2.8984364e-06, ..., 2.9640032e-06,
        6.0490436e-07, 1.9536097e-05], dtype=float32))

In [31]:
# predicting 4th word & its probability
word_idx = 3
vocab_id = np.argmax(next_word_preds[0][word_idx-1])
tokenizer.convert_ids_to_tokens([vocab_id]) , next_word_preds[0][word_idx-1].numpy()

(['Ġgoing'],
 array([3.1142408e-04, 5.4542674e-05, 1.7687354e-06, ..., 1.4735940e-07,
        8.5331287e-08, 5.6419531e-06], dtype=float32))

In [32]:
# predicting 5th word & its probability
word_idx = 4
vocab_id = np.argmax(next_word_preds[0][word_idx-1])
tokenizer.convert_ids_to_tokens([vocab_id]) , next_word_preds[0][word_idx-1].numpy()

(['Ġthis'],
 array([2.0381255e-04, 7.8330733e-05, 1.7114537e-06, ..., 4.4928004e-07,
        1.4243042e-08, 1.1868745e-05], dtype=float32))

In [56]:
# predicting 6th word & its probability
word_idx = 5
vocab_id = np.argmax(next_word_preds[0][word_idx-1])
tokenizer.convert_ids_to_tokens([vocab_id]) , next_word_preds[0][word_idx-1].numpy()

(['?'], array([4.6291811e-04, 1.5614973e-04, 8.9703753e-06, ..., 3.8205088e-09,
        4.8937618e-09, 2.2715134e-05], dtype=float32))

**This is how generated text would've been like for different lengths of above input prefix**

In [55]:
print(f"{generator('how', do_sample=False, pad_token_id=tokenizer.eos_token_id )[0]['generated_text'].strip()}")

print(f"{generator('how are', do_sample=False, pad_token_id=tokenizer.eos_token_id)[0]['generated_text'].strip()}")

print(f"{generator('how are you', do_sample=False, pad_token_id=tokenizer.eos_token_id)[0]['generated_text'].strip()}")

print(f"{generator('how are you doing', do_sample=False, pad_token_id=tokenizer.eos_token_id)[0]['generated_text'].strip()}")

print(f"{generator('how are you doing today', do_sample=False, pad_token_id=tokenizer.eos_token_id)[0]['generated_text'].strip()}")

how The New York Times has been reporting on the Trump administration’s decision to fire former FBI Director James Comey.
how are you going to do that?”
how are you going to do that?”
how are you doing this?
how are you doing today?


#### Exploring its word token and position embeddings

In [61]:
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

In [63]:
word_embeddings = model.transformer.wte.weight  # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings 

In [64]:
word_embeddings.shape

torch.Size([50257, 768])

In [65]:
position_embeddings.shape

torch.Size([1024, 768])

### References
* [Language Models are unsupervised multitask learners](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) 
* https://huggingface.co/transformers/v3.0.2/model_doc/gpt2.html#gpt2tokenizerfast
* https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2LMHeadModel
* https://huggingface.co/docs/transformers/tokenizer_summary
* [Open AI GPT, 2018](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf)